In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib
from localtsfresh.tsfresh.examples.har_dataset import download_har_dataset, load_har_dataset, load_har_classes
# import seaborn as sns
from localtsfresh.tsfresh import extract_features, extract_relevant_features, select_features
from localtsfresh.tsfresh.utilities.dataframe_functions import impute
from localtsfresh.tsfresh.feature_extraction import ComprehensiveFCParameters, EfficientFCParameters, MinimalFCParameters
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from pandas import DataFrame
import pandas as pd
import numpy as np
import os
import math

import logging

# We set the logger to Error level
# This is not recommend for normal use as you can oversee important Warning messages
logging.basicConfig(level=logging.ERROR)

# 六种出行方式
travel=["bus", "car", "running", "stationary", "subway", "walking"]
# 六个传感器数据
sensor=["azimath", "pitch", "roll", "north", "east", "up"]
# 一些常量
filenum=6
sensornum=6
#窗口大小
win=50
#使用的样本数量(一个窗口的数据是一个样本)，建议设置陈6的倍数
use_num=600

#根据Win，生成DataFrame格式中的Time列
def getTimeColumn(win):
    time=np.arange(win)
    for idx in range(1, sensornum):
        timetmp=np.arange(win)
        time=np.concatenate((time, timetmp), axis=0)
    time.shape=(len(time),1)
    return time

#根据Win和数据，生成DataFrame格式中的kind和value列
def getKindValueColumn(sd, win):
    for i in range(sensornum):
        kind=i*np.ones((win, 1),dtype=int)
        sensorcols=sd[i]
        sensorcols.shape=(win, 1)
        sdata=np.column_stack((kind, sensorcols))
        if i==0:
            sensorframe=sdata
        else:
            sensorframe=np.row_stack((sensorframe, sdata))
    return sensorframe

def getIdColumn(num, win):
    for i in range(num):
        if i==0:
            idary=0*np.ones((sensornum*win, 1))
        else:
            idary=np.row_stack((idary, i*np.ones((sensornum*win, 1))))
    return idary

def getDataLabelColumn(dataary,label, num, win):
    for i in range(filenum):
        if i==0:
            y=label[travel[i]][:num]
            data=dataary[travel[i]][:num*win*sensornum]
        else:
            y=np.row_stack((y, label[travel[i]][:num]))
            data=np.row_stack((data, dataary[travel[i]][:num*win*sensornum]))
    y.shape=(len(y),)
    y=pd.Series(y)
    return data, y

#读取所有数据，结果是字典，分别存储六种运动的传感器读数矩阵，每个矩阵的三列分别是time, kind, value
def loadData(win):
    labeldict={}
    datadict={}
    files = os.listdir('/home/hadoop/data')
    for i in range(0, filenum):
        with open('/home/hadoop/data/%s' % files[i], 'r') as f:
            sensordata=np.loadtxt(f, delimiter=",")
            for j in range(0, math.ceil(len(sensordata)/win)):
                sd=sensordata[win*j:win*(j+1),:]
                sd=sd.transpose()
                
                time=getTimeColumn(win)
                kindvalue=getKindValueColumn(sd, win)
                travary=np.column_stack((time, kindvalue ))
                
                if j == 0:
                    dataarray=travary
                else:
                    dataarray=np.concatenate((dataarray, travary), axis=0)  

            labeldict[travel[i]]=i*np.ones((math.ceil(len(sensordata)/win), 1), dtype=int)
            datadict[travel[i]]=dataarray
            print(files[i]+" loaded!")
            
    return datadict, labeldict

#从所有数据dataary和其标签label中，选出总数为num的样本，每个类别选num/sensornum个样本。
def genTrainSample(dataary, label, num, win):
    idary=getIdColumn(num, win)
    labelnum=int(num/sensornum) ;
    data, y=getDataLabelColumn(dataary, label, labelnum, win)
    data=np.column_stack((idary, data))
    dataframe = DataFrame(data, columns=['id', 'time', 'kind', 'value'])
    return dataframe, y


localtsfresh_init!
feature_extraction_init
tsfresh_init!


In [2]:
data, label = loadData(win)
df, y=genTrainSample(data, label, use_num, win) 
master_df = df

print(master_df.shape)

bus_data_set_101 loaded!
car_data_set_103 loaded!
running_data_set_8 loaded!
stationary_data_set_3 loaded!
subway_data_set_102 loaded!
walking_data_set_7 loaded!
(180000, 4)


In [3]:
fc_parameters = {
    "fft_topk_freqs":None,
    
    
}
print fc_parameters
#testfunc()

{'fft_topk_freqs': None}


In [4]:
from localtsfresh.tsfresh import extract_features, extract_relevant_features, select_features
%time X = extract_features(master_df, default_fc_parameters=fc_parameters,column_id='id', column_sort="time", column_kind="kind", column_value="value")

It is ok!


Feature Extraction: 100%|██████████| 3600/3600 [00:01<00:00, 2378.25it/s]

CPU times: user 1.04 s, sys: 73.9 ms, total: 1.12 s
Wall time: 2.2 s


In [10]:
impute(X)
X.shape

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [9]:
X(1,1).type

TypeError: 'DataFrame' object is not callable